## Dados demográficos (SIDRA - 6706)
Para acessar dados do Sistema IBGE de Recuperação de Estatísticas (SIDRA) use a função sidrapy.get_table para aceder à tabela.
os argumentos são: 
    table_code: str, código da tabela. e.g. table_code='6706', 
    territorial_level: str, nível territorial, '1' para Brasil, '2' para Grande região, '3' para estado, '7' para RM, '6' para município.
    ibge_territorial_code: str, consulta em (http://api.sidra.ibge.gov.br/) informando o número da tabela. Recife responde por '2611606'
    variable: opcional. no exemplo, '606', população em mil pessoas.
    classifications: Optional[Dict[str, str]] = None - classificações e suas categorias. ex: "2": "6794,5" Sexo, total e mulheres.
    period: Optional[str] = None, sem preenchimento retorna resultado mais recente
    header: Optional[str] = None, ="y" 
    format: str = 'pandas'# 

In [ ]:
import sidrapy 

cidade = "Nome da cidade"
cod_cidade = "#######"

regiaometropolitana = "nome da região"
cod_regiaometropolitana = "####"

estado = "Nome do estado"
cod_estado = "##"

granderegiao = "nome da região"
cod_granderegiao = "#"

# Cidade
data_cidade = sidrapy.get_table(
    table_code= "6706",
    territorial_level= "6",
    ibge_territorial_code= cod_cidade,
    variable = "606",
    classifications={"2": "6794,5", "58": "all"},
    header="y",
    format= 'pandas'
)

# Região Metropolitana
data_regiaometropolitana = sidrapy.get_table(
    table_code= "6706",
    territorial_level= "7",
    ibge_territorial_code= cod_regiaometropolitana,
    variable = "606",
    classifications={"2": "6794,5", "58": "all"},
    header="y",
    format= 'pandas'
)
# Estado
data_estado = sidrapy.get_table(
    table_code= "6706",
    territorial_level= "3",
    ibge_territorial_code= cod_estado,
    variable = "606",
    classifications={"2": "6794,5", "58": "all"},
    header="y",
    format= 'pandas'
)
# Grande Região
data_granderegiao = sidrapy.get_table(
    table_code= "6706",
    territorial_level= "2",
    ibge_territorial_code= cod_granderegiao,
    variable = "606",
    classifications={"2": "6794,5", "58": "all"},
    header="y",
    format= 'pandas'
)
#Brasil
data_brasil = sidrapy.get_table(
    table_code= "6706",
    territorial_level= "1",
    ibge_territorial_code= "1",
    variable = "606",
    classifications={"2": "6794,5", "58": "all"},
    header="y",
    format= 'pandas'
)

# Criando lista com dataframes
dfs = [data_cidade, data_regiaometropolitana, data_estado, data_granderegiao, data_brasil]

for df in dfs:
  # Modificando header
    df.columns = df.iloc[0] 
    df.drop(0, inplace=True)
# Aplicando cast na coluna valor    
    df['Valor'] = df['Valor'].astype(int)

# Definindo grupos de idade
jovens = ['15 a 19 anos','20 a 24 anos','25 a 29 anos']
pessoas_idosas = ['60 a 64 anos','65 a 69 anos','70 a 74 anos','75 a 79 anos','80 anos ou mais']

# Filtrando sexo por todos e somando a população de todas as faixas etárias (população total)
def soma_populacao(df):
    return df[df['Sexo']=='Total']['Valor'].sum()

# Filtrando grupos prioritários e apresentando sua proporção em relação à população total

def percentual_mulheres(df):
    return (df[df['Sexo']=='Mulheres']['Valor'].sum() / soma_populacao(df)) * 100

def percentual_jovens(df):
    return (df[(df['Sexo']=='Total')&(df['Grupo de idade'].isin(jovens))]['Valor'].sum()/soma_populacao(df)) * 100

def percentual_pessoas_idosas(df):
    return (df[(df['Sexo']=='Total')&(df['Grupo de idade'].isin(pessoas_idosas))]['Valor'].sum()/soma_populacao(df)) * 100



## Atribuindo variáveis aos resultados

Após garantir a correta extração dos dados em todos os níveis territoriais, devemos atribuir variáveis para os respectivos dados, de forma a possibilitar a montagem de uma planilha capaz de automatizar o nosso relatório.

In [ ]:
#Populacao total

pop_cidade = soma_populacao(data_cidade)
pop_regiaometropolitana = soma_populacao(data_regiaometropolitana)
pop_estado = soma_populacao(data_estado)
pop_granderegiao = soma_populacao(data_granderegiao)
pop_br = soma_populacao(data_brasil)

# Percentual mulheres

mulheres_cidade = percentual_mulheres(data_cidade)
mulheres_regiaometropolitana = percentual_mulheres(data_regiaometropolitana)
mulheres_estado = percentual_mulheres(data_estado)
mulheres_granderegiao = percentual_mulheres(data_granderegiao)
mulheres_br = percentual_mulheres(data_brasil)

# Percentual jovens

jovens_cidade = percentual_jovens(data_cidade)
jovens_regiaometropolitana = percentual_jovens(data_regiaometropolitana)
jovens_estado= percentual_jovens(data_estado)
jovens_granderegiao = percentual_jovens(data_granderegiao)
jovens_br = percentual_jovens(data_brasil)

# Percentual pessoas idosas

pessoas_idosas_cidade = percentual_pessoas_idosas(data_cidade)
pessoas_idosas_regiaometropolitana = percentual_pessoas_idosas(data_regiaometropolitana)
pessoas_idosas_estado= percentual_pessoas_idosas(data_estado)
pessoas_idosas_granderegiao = percentual_pessoas_idosas(data_granderegiao)
pessoas_idosas_br = percentual_pessoas_idosas(data_brasil)


## Imprimir resultados para visualização

In [ ]:
# Imprimindo os resultados para verificação

print(f'''

População

{cidade}: {soma_populacao(data_cidade)/1000:.2f} milhões
{regiaometropolitana}: {soma_populacao(data_regiaometropolitana)/1000:.2f} milhões
{estado}: {soma_populacao(data_estado)/1000:.2f} milhões
{granderegiao} {soma_populacao(data_granderegiao)/1000:.2f} milhões
BR: {soma_populacao(data_brasil)/1000:.1f} milhões

Percentual mulheres

{cidade}: {percentual_mulheres(data_cidade):.1f}%
{regiaometropolitana}: {percentual_mulheres(data_regiaometropolitana):.1f}%
{estado}: {percentual_mulheres(data_estado):.1f}%
{granderegiao} {percentual_mulheres(data_granderegiao):.1f}%
BR: {percentual_mulheres(data_brasil):.1f}%

Percentual jovens

{cidade}: {percentual_jovens(data_cidade):.1f}%
{regiaometropolitana}: {percentual_jovens(data_regiaometropolitana):.1f}%
{estado}: {percentual_jovens(data_estado):.1f}%
{granderegiao} {percentual_jovens(data_granderegiao):.1f}%
BR: {percentual_jovens(data_brasil):.1f}%

Percentual pessoas idosas

{cidade}: {percentual_pessoas_idosas(data_cidade):.1f}%
{regiaometropolitana}: {percentual_pessoas_idosas(data_regiaometropolitana):.2f}%
{estado}: {percentual_pessoas_idosas(data_estado):.1f}%
{granderegiao} {percentual_pessoas_idosas(data_granderegiao):.1f}%
BR: {percentual_pessoas_idosas(data_brasil):.1f}%
''')




## Enviando resultados por e-mail

In [ ]:
from email.message import EmailMessage
import os
import smtplib
from dotenv import load_dotenv

load_dotenv()

email = os.environ.get("email")
senha = os.environ.get("senha")

msg = EmailMessage()
msg['Subject'] = 'Dados Demográficos com o Python'
msg['From'] = email
msg['To'] = 'destinatario@email.com' #inserir destinatário


msg.set_content(f'''
Prezados, segue abaixo os dados requisitados referente a população, % de mulheres, % de jovens, % de idosos do {cidade}, {regiaometropolitana}, {estado}, {granderegiao} e Brasil.

População

{cidade}: {soma_populacao(data_cidade)/1000:.2f} milhões
{regiaometropolitana}: {soma_populacao(data_regiaometropolitana)/1000:.2f} milhões
{estado} {soma_populacao(data_estado)/1000:.2f} milhões
{granderegiao}: {soma_populacao(data_granderegiao)/1000:.2f} milhões
BR: {soma_populacao(data_brasil)/1000:.1f} milhões

Percentual mulheres

{cidade}: {percentual_mulheres(data_cidade):.1f}%
{regiaometropolitana}: {percentual_mulheres(data_regiaometropolitana):.1f}%
{estado} {percentual_mulheres(data_estado):.1f}%
{granderegiao}: {percentual_mulheres(data_granderegiao):.1f}%
BR: {percentual_mulheres(data_brasil):.1f}%

Percentual jovens

{cidade}: {percentual_jovens(data_cidade):.1f}%
{regiaometropolitana}: {percentual_jovens(data_regiaometropolitana):.1f}%
{estado} {percentual_jovens(data_estado):.1f}%
{granderegiao}: {percentual_jovens(data_granderegiao):.1f}%
BR: {percentual_jovens(data_brasil):.1f}%

Percentual pessoas idosas

{cidade}: {percentual_pessoas_idosas(data_cidade):.1f}%
{regiaometropolitana}: {percentual_pessoas_idosas(data_regiaometropolitana):.1f}%
{estado} {percentual_pessoas_idosas(data_estado):.1f}%
{granderegiao}: {percentual_pessoas_idosas(data_granderegiao):.1f}%
BR: {percentual_pessoas_idosas(data_brasil):.1f}%

Abraços,
<seu nome>
''')

with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
    
    smtp.login(email, senha)
    smtp.send_message(msg)